In [ ]:
# Import needed libraries
import pandas as pd

### This is for loading the data

# The crawl dataset we are using do not have column names or headings so we need to handle that
column_names = ['videoID', 'uploader', 'age', 'category', 'length', 'views', 'rate', 'ratings', 'comments', 'relatedIDs']

# Load YouTube data for each depth file(we have 4 depths in total starting from 0)
def load_depth(path, crawl_date, depth):
    # Initialize a list to store the valid rows
    valid_rows = []
    # Read our dataset line by line
    # We were using pandas to load/read data at first but there were errors when the 1st row only has 1 or 2 column and pandas were assuming that's all the columns we have (inconsistent)
    # so we decided to go with this route instead and it works
    with open(path, 'r') as file:
        for line in file:
            # Split the line by tab and check if it has at least 2 columns
            # Some rows might have only 1 or 2 columns which is not useful for us, so we ignore them
            # this way it helps with fixing the issues with some rows having only 1 column as well
            split_line = line.strip().split('\t')
            if len(split_line) >= 2:
                valid_rows.append(split_line)       
    # Convert the list into pandas dataframe
    depth_data = pd.DataFrame(valid_rows)
    depth_data['crawl_date'] = crawl_date
    depth_data['depth'] = depth
    return depth_data

# Load all depth files for a single crawl
def load_crawl(path, crawl_date):
    # Load depth file 0, 1, 2 and 3
    depth_files = [f"{path}/{i}.txt" for i in range(4)]
    depth_dframe = []
    # Loop through the total depth files and add the dataframe to depth_dframe list
    for i, depth_file in enumerate (depth_files):
        depth_dframe.append(load_depth(depth_file, crawl_date, i))
    # Now for each crawl, we combine all depths into a single dataframe
    combined_data = pd.concat(depth_dframe, axis=0).reset_index(drop = True)
    return combined_data

In [ ]:
### This is for cleaning and transformation the data (Data Preparation)
def prepare_data(df):
    # Since there may be more than 1 related ids, we handle it by combining the related IDs together as a single list
    combined_related_ids = []
    # Loop over each row and combine the related IDs
    for index, row in df.iterrows():
        # We select the related IDs columns (from the 10th column onward) 
        # The dataset description says the related IDs is up to 20 strings only 
        related_ids = row[9:29]  
        # Join the cleaned related IDs into a single string separated by commas then add to the list
        combined_related_ids.append(','.join(related_ids.astype(str)))
    # Add the combined relatedIDs to the DataFrame
    df['relatedIDs'] = combined_related_ids
    # Keep only the first 9 columns plus the new combined 'relatedIDs' column
    depth_data = df.iloc[:, :9].copy() 
    depth_data['relatedIDs'] = combined_related_ids 
    # Keep the crawl_date and depth column 
    depth_data[['crawl_date', 'depth']] = df[['crawl_date', 'depth']] 
    # Add the column names to each column
    depth_data.columns = column_names + ['crawl_date', 'depth']
    # Remove the leading/trailing whitespace from string columns
    depth_data['uploader'] = depth_data['uploader'].str.strip()
    depth_data['category'] = depth_data['category'].str.strip()
    # Convert these columns to numeric
    numeric_columns = ['age', 'length', 'views', 'rate', 'ratings', 'comments']
    for col in numeric_columns:
        depth_data[col] = pd.to_numeric(depth_data[col])   
    # Fill in the missing 'rate' values with the mean of the column
    depth_data['rate'] = depth_data['rate'].fillna(depth_data['rate'].mean())
    return depth_data

In [ ]:
# testing = (load_depth("data/080331/2.txt", '2008-03-31', "2"))
# clean = clean_data(testing)
# print(clean)

# Load all the crawls in the given dataset
crawl1 = load_crawl('data/080327', '2008-03-27')
crawl2 = load_crawl('data/080329', '2008-03-29')
crawl3 = load_crawl('data/080331', '2008-03-31')
# Combine all crawls into a single dataframe
combined_data = pd.concat([crawl1, crawl2, crawl3], axis=0).reset_index(drop=True)
# Prepare the data (clean & transform)
combined_data = prepare_data(combined_data)
print(combined_data.head())

In [ ]:
# Data ingestion and connection with MongoDB
from pymongo import MongoClient
# Connect to MongoDb database that we have created
uri = 'mongodb://localhost:27017/'
client = MongoClient(uri)
db = client["youtubedb"]
collection = db["youtube_vids"]
# Convert the DataFrame into a list of dictionaries
insert_data = combined_data.to_dict('records')
# Insert the data into the collection
collection.insert_many(insert_data)

In [ ]:
# Validation Queries

# Query 1: Validate Document Count
document_count = collection.count_documents({})
print("Total documents:", document_count)

# Query 2: Check Category Distribution
category_counts = collection.aggregate([
    { "$group": { "_id": "$category", "count": { "$sum": 1 } } }
])
print("Category distribution:")
for category in category_counts:
    print(category)

# Query 3: Check Average and Max Views
view_stats = collection.aggregate([
    { "$group": { "_id": None, "avgViews": { "$avg": "$views" }, "maxViews": { "$max": "$views" } } }
])
print("View statistics:")
for stats in view_stats:
    print(stats)

# Query 4: Validate Related IDs Format
related_id_check = collection.find({"relatedIDs": {"$exists": True, "$type": "string"}}).limit(5)
print("Sample related IDs:")
for doc in related_id_check:
    print(doc['relatedIDs'])


In [ ]:
import time

# Clear the collection to avoid duplicate entries
collection.delete_many({})
print("Collection cleared.")

# Start timer
start_time = time.time()

# Ingest data (insert many records)
collection.insert_many(insert_data)

# End timer
end_time = time.time()

# Calculate and print ingestion time
ingestion_time = end_time - start_time
print(f"Data ingestion took {ingestion_time:.2f} seconds")


In [ ]:
import time

# Measure execution time for document count
start_time = time.time()
document_count = collection.count_documents({})
end_time = time.time()

# Print results
print("Total documents:", document_count)
print(f"Document count query took {end_time - start_time:.4f} seconds")


In [ ]:
# Measure execution time for category distribution query
start_time = time.time()
category_counts = collection.aggregate([
    { "$group": { "_id": "$category", "count": { "$sum": 1 } } }
])

# Collect results to ensure the query executes fully
categories = list(category_counts)
end_time = time.time()

# Print results and time
print("Category distribution:")
for category in categories:
    print(category)
print(f"Category distribution query took {end_time - start_time:.4f} seconds")


In [ ]:
# Measure execution time for view statistics query
start_time = time.time()
view_stats = collection.aggregate([
    { "$group": { "_id": None, "avgViews": { "$avg": "$views" }, "maxViews": { "$max": "$views" } } }
])

# Collect results to ensure the query executes fully
view_stats_result = list(view_stats)
end_time = time.time()

# Print results and time
print("View statistics:", view_stats_result)
print(f"View statistics query took {end_time - start_time:.4f} seconds")


In [ ]:
# Measure execution time for related IDs format check
start_time = time.time()
related_id_check = collection.find({"relatedIDs": {"$exists": True, "$type": "string"}}).limit(5)

# Collect results to ensure the query executes fully
related_ids = list(related_id_check)
end_time = time.time()

# Print results and time
print("Sample related IDs:")
for doc in related_ids:
    print(doc['relatedIDs'])
print(f"Related IDs check query took {end_time - start_time:.4f} seconds")
